In [ ]:
#SCRIpT 1 : Simulasi Rotasi Baton, angular velocity constan
import tkinter as tk
import math
from vector import Vector 

# Class Baton (tongkat)
class Baton:
    def __init__(self, canvas, center, length):
        self.canvas = canvas
        self.center = center                # Vector posisi tengah
        self.length = length                # Panjang tongkat
        self.angle_deg = 0                  # Sudut awal (dalam derajat)
        self.angular_velocity = 2           # Kecepatan sudut (derajat/frame)
        self.angular_acceleration = 0       # Percepatan sudut (tetap nol)
        
        # Garis tongkat di canvas
        self.line = canvas.create_line(0, 0, 0, 0, width=8, fill="blue")
        
        # Teks informasi di sudut layar
        self.text = canvas.create_text(10, 10, anchor="nw", text="", font=("Arial", 12), fill="black")

    def update(self):
        # Konversi sudut ke radian
        angle_rad = math.radians(self.angle_deg)

        # Buat arah unit vector berdasarkan sudut
        direction = Vector(math.cos(angle_rad), math.sin(angle_rad))

        # Hitung dua ujung tongkat berdasarkan pusat
        end1 = self.center.added(direction.multed(self.length))
        end2 = self.center.subbed(direction.multed(self.length))

        # Perbarui garis tongkat pada canvas
        self.canvas.coords(self.line, end1.x, end1.y, end2.x, end2.y)

        # Tampilkan informasi perhitungan pada layar
        info = f"""Sudut (derajat): {self.angle_deg:.1f}°
Sudut (radian): {angle_rad:.2f}
Angular Velocity: {self.angular_velocity:.2f}°/frame
Angular Acceleration: {self.angular_acceleration:.2f}°/frame²
Ujung 1: ({end1.x:.1f}, {end1.y:.1f})
Ujung 2: ({end2.x:.1f}, {end2.y:.1f})
"""
        self.canvas.itemconfig(self.text, text=info)

        # Update sudut dengan angular velocity
        self.angle_deg = (self.angle_deg + self.angular_velocity) % 360


# Inisialisasi Tkinter dan kanvas
root = tk.Tk()
root.title("Simulasi Rotasi Baton - Nature of Code 3.1")

canvas = tk.Canvas(root, width=600, height=400, bg="white")
canvas.pack()

# Titik tengah canvas sebagai Vector
center = Vector(300, 200)

# Buat objek Baton
baton = Baton(canvas, center, length=100)

# Fungsi animasi
def animate():
    baton.update()
    root.after(50, animate)

animate()
root.mainloop()

In [ ]:
#SCRIPT 2 : Simulasi Rotasi Baton, angular_velocity = 0.05, angular_acceleration = 0.001 
import tkinter as tk
import math
from vector import Vector

# ==========================
# Class Baton (Tongkat)
# ==========================
class Baton:
    def __init__(self, center, length):
        self.center = center                # Titik pusat rotasi (Vector)
        self.length = length               # Panjang tongkat

        self.angle = 0                     # Sudut awal (radian)
        self.angular_velocity = 0.05       # Kecepatan sudut
        self.angular_acceleration = 0.0005 # Percepatan sudut

        self.line = None                   # ID garis pada canvas
        self.circle1 = None                # Lingkaran ujung kiri
        self.circle2 = None                # Lingkaran ujung kanan

    def update(self):
        # Update sudut dan kecepatan sudut
        self.angle += self.angular_velocity
        self.angular_velocity += self.angular_acceleration

    def display(self, canvas):
        # Hitung arah rotasi sebagai unit vector
        direction = Vector(math.cos(self.angle), math.sin(self.angle))
        half = direction.multed(self.length / 2)

        # Ujung kiri dan kanan dari pusat
        end1 = self.center.subbed(half)
        end2 = self.center.added(half)

        if self.line is None:
            # Pertama kali buat objek canvas
            self.line = canvas.create_line(end1.x, end1.y, end2.x, end2.y, width=4, fill="blue")
            self.circle1 = canvas.create_oval(end1.x-6, end1.y-6, end1.x+6, end1.y+6, fill="red")
            self.circle2 = canvas.create_oval(end2.x-6, end2.y-6, end2.x+6, end2.y+6, fill="red")
        else:
            # Update posisi objek
            canvas.coords(self.line, end1.x, end1.y, end2.x, end2.y)
            canvas.coords(self.circle1, end1.x-6, end1.y-6, end1.x+6, end1.y+6)
            canvas.coords(self.circle2, end2.x-6, end2.y-6, end2.x+6, end2.y+6)

        return self.angle, self.angular_velocity, self.angular_acceleration


# ==========================
# Fungsi Update Layar
# ==========================
def update():
    baton.update()
    angle, a_vel, a_acc = baton.display(canvas)

    # Tampilkan informasi di layar
    info_text = (
        f"Angle (rad) = {angle:.3f}\n"
        f"Angular Velocity = {a_vel:.3f}\n"
        f"Angular Acceleration = {a_acc:.5f}"
    )
    label_info.config(text=info_text)

    window.after(30, update)  # Loop animasi


# ==========================
# Setup Tkinter Window
# ==========================
WIDTH, HEIGHT = 600, 400
window = tk.Tk()
window.title("Simulasi Rotasi Sudut - The Nature of Code 3.2")

canvas = tk.Canvas(window, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

label_info = tk.Label(window, text="", font=("Arial", 12), justify="left")
label_info.pack()

# Buat baton di tengah layar
center = Vector(WIDTH / 2, HEIGHT / 2)
baton = Baton(center, length=150)

# Mulai animasi
update()
window.mainloop()


In [12]:
# SCRIPT 3 : # Simulasi Gaya Tarik dengan Gerakan Angular
import tkinter as tk
import math
import random
from vector import Vector  

#1. Fungsi ini memastikan nilai tetap dalam batas tertentu. Contoh:
#constrain(10, 0, 5) akan mengembalikan 5
#constrain(-2, 0, 5) akan mengembalikan 0
def constrain(val, min_val, max_val):
    return max(min_val, min(val, max_val))
#2. Bola Merah (Attractor)
class Attractor:
    def __init__(self, canvas, x=320, y=120):
        self.canvas = canvas
        self.position = Vector(x, y)  # Posisi tengah layar
        self.mass = 20                # Massa besar
        self.G = 1                    # Kekuatan gravitasi
        r = self.mass * 2             # Ukuran bola
        # Gambar lingkaran merah
        self.id = canvas.create_oval(x-r, y-r, x+r, y+r, fill="red")
    # Attractor mengaplikasikan gaya tarik pada Mover
    def attract(self, mover):
        # Hitung jarak antara Attractor dan Mover
        force = Vector(self.position.x - mover.position.x, 
                    self.position.y - mover.position.y)
        distance = force.mag()  # Panjang garis antara keduanya
        
        # Batasi jarak minimal 5 dan maksimal 25
        distance = constrain(distance, 5, 25)
        
        force.normalize()  # Buat vektor panjangnya menjadi 1
        
        # Hitung kekuatan tarik (seperti rumus gravitasi)
        strength = (self.G * self.mass * mover.mass) / (distance * distance)
        force.mult(strength)  # Terapkan kekuatan ke vektor
        
        return force

    def display(self):
        r = self.mass * 2
        self.canvas.coords(self.id, self.position.x - r, self.position.y - r,
                                  self.position.x + r, self.position.y + r)
#3. Bola Kecil (Mover)
class Mover:
    def __init__(self, canvas, x, y, mass):
        self.canvas = canvas
        self.mass = mass
        self.radius = self.mass * 8  # Semakin besar massa, semakin besar ukuran
        self.position = Vector(x, y) # Posisi acak
        # Kecepatan awal acak antara -1 sampai 1
        self.velocity = Vector(random.uniform(-1, 1), random.uniform(-1, 1))
        self.acceleration = Vector(0, 0)

        self.angle = 0
        self.angle_velocity = 0
        self.angle_acceleration = 0

        # Gambar bola abu-abu dengan garis penunjuk arah
        r = self.radius
        self.id_circle = canvas.create_oval(x-r, y-r, x+r, y+r, fill="gray")
        self.id_line = canvas.create_line(x, y, x+r, y, width=2, fill="black")

    def apply_force(self, force):
        f = force.copy()
        f.div(self.mass)
        self.acceleration.add(f)
    #Pergerakan:
    def update(self):
        # Update posisi berdasarkan kecepatan
        self.velocity.add(self.acceleration)
        self.position.add(self.velocity)

        # Putar garis penunjuk berdasarkan percepatan
        self.angle_acceleration = self.acceleration.x / 10.0
        self.angle_velocity += self.angle_acceleration
        self.angle_velocity = constrain(self.angle_velocity, -0.1, 0.1)
        self.angle += self.angle_velocity

        self.acceleration.mult(0)  # Reset percepatan
    #Gambar Ulang Posisi:
    def show(self):
        x, y, r = self.position.x, self.position.y, self.radius
        # Update posisi lingkaran
        self.canvas.coords(self.id_circle, x-r, y-r, x+r, y+r)
        
        # Hitung ujung garis penunjuk
        end_x = x + r * math.cos(self.angle)
        end_y = y + r * math.sin(self.angle)
        # Update posisi garis
        self.canvas.coords(self.id_line, x, y, end_x, end_y)

# ========================
# Main Program
# ========================
root = tk.Tk()
root.title("Example 3.2: Forces with Angular Motion - Nature of Code")

canvas = tk.Canvas(root, width=640, height=240, bg="white")
canvas.pack()

attractor = Attractor(canvas)

# Buat 20 bola kecil dengan posisi dan massa acak
movers = []
for _ in range(20):
    x = random.uniform(0, 640)  # Posisi X acak
    y = random.uniform(0, 240)  # Posisi Y acak
    mass = random.uniform(0.1, 2)  # Massa acak
    movers.append(Mover(canvas, x, y, mass))

def animate():
    attractor.display()
    for mover in movers:
        force = attractor.attract(mover)  # Hitung gaya tarik
        mover.apply_force(force)         # Terapkan gaya
        mover.update()                   # Update posisi
        mover.show()                     # Gambar ulang
    root.after(30, animate)             # Ulangi setiap 30ms

animate()
root.mainloop()


In [ ]:
#SCRIPT 4 : # Simulasi Peluru Meriam dengan Rotasi
import tkinter as tk
import math
from vector import Vector  

class CannonBall:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)
        self.r = 8
        self.topspeed = 10
        self.id = canvas.create_oval(x - self.r, y - self.r, x + self.r, y + self.r, fill="black")

    def apply_force(self, force):
        self.acceleration.add(force)

    def update(self):
        self.velocity.add(self.acceleration)
        self.velocity.limit(self.topspeed)
        self.position.add(self.velocity)
        self.acceleration.mult(0)

    def display(self):
        x, y, r = self.position.x, self.position.y, self.r
        self.canvas.coords(self.id, x - r, y - r, x + r, y + r)

class Cannon:
    def __init__(self, canvas, x, y):
        self.canvas = canvas
        self.position = Vector(x, y)
        self.angle = -math.pi / 4  # -45 derajat
        self.length = 50
        self.id = None

    def draw(self):
        x, y = self.position.x, self.position.y
        end_x = x + self.length * math.cos(self.angle)
        end_y = y + self.length * math.sin(self.angle)
        if self.id is None:
            self.id = self.canvas.create_line(x, y, end_x, end_y, width=10, fill="gray")
        else:
            self.canvas.coords(self.id, x, y, end_x, end_y)

    def rotate(self, delta_angle):
        self.angle += delta_angle
        self.angle = max(-math.pi / 2, min(self.angle, 0))  # constrain sudut

    def shoot_force(self, magnitude=10):
        force = Vector(math.cos(self.angle), math.sin(self.angle))
        force.mult(magnitude)
        return force

# Setup Tkinter
root = tk.Tk()
root.title("Exercise 3.2 Cannon - Nature of Code")

WIDTH, HEIGHT = 640, 360
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

cannon = Cannon(canvas, 50, 300)
ball = CannonBall(canvas, cannon.position.x, cannon.position.y)
shot = False

def animate():
    global ball, shot
    canvas.delete("background_text")

    cannon.draw()

    if shot:
        gravity = Vector(0, 0.2)
        ball.apply_force(gravity)
        ball.update()

    ball.display()

    # Reset jika jatuh
    if ball.position.y > HEIGHT:
        ball = CannonBall(canvas, cannon.position.x, cannon.position.y)
        shot = False

    # Tampilkan informasi di layar
    angle_deg = math.degrees(cannon.angle)
    canvas.create_text(10, 10, anchor="nw", text=f"Angle: {angle_deg:.1f}°", tag="background_text", font=("Arial", 14))

    pos = ball.position
    vel = ball.velocity
    acc = ball.acceleration
    canvas.create_text(10, 30, anchor="nw", text=f"Pos: ({pos.x:.1f}, {pos.y:.1f})", tag="background_text", font=("Arial", 12))
    canvas.create_text(10, 50, anchor="nw", text=f"Vel: ({vel.x:.2f}, {vel.y:.2f})", tag="background_text", font=("Arial", 12))
    canvas.create_text(10, 70, anchor="nw", text=f"Acc: ({acc.x:.2f}, {acc.y:.2f})", tag="background_text", font=("Arial", 12))

    root.after(30, animate)

def on_key_press(event):
    global shot
    if event.keysym == 'Left':
        cannon.rotate(-0.1)
    elif event.keysym == 'Right':
        cannon.rotate(0.1)
    elif event.keysym == 'space' and not shot:
        shot = True
        force = cannon.shoot_force(10)
        ball.apply_force(force)

root.bind("<KeyPress>", on_key_press)

animate()
root.mainloop()


In [ ]:
#SCRIPT 5 : # Simulasi mobil kotak yang bergerak mengikuti arah mouse
import tkinter as tk
import math
from vector import Vector

class Mover:
    def __init__(self, canvas, width, height):
        self.canvas = canvas  # tempat menggambar
        self.width = width    # lebar layar
        self.height = height  # tinggi layar
        self.position = Vector(width / 2, height / 2)  # posisi awal di tengah
        self.velocity = Vector(0, 0)  # kecepatan awal (diam)
        self.acceleration = Vector(0, 0)  # percepatan awal
        self.topspeed = 4  # kecepatan maksimum
        self.r = 16  # ukuran
        self.id = canvas.create_polygon(0, 0, 0, 0, 0, 0, fill="gray", outline="black")  # bentuk kotak

    def update(self, mouse_x, mouse_y):
        mouse = Vector(mouse_x, mouse_y)  # posisi mouse
        dir = mouse.subbed(self.position)    # arah menuju mouse
        dir.normalize()  # buat panjang vektor = 1
        dir.mult(0.5)    # perkecil pengaruhnya
        self.acceleration = dir  # set percepatan
        
        self.velocity.add(self.acceleration)  # tambah kecepatan
        self.velocity.limit(self.topspeed)    # batasi kecepatan maks
        self.position.add(self.velocity)      # update posisi

    def display(self):
        angle = self.velocity.heading()  # sudut berdasarkan arah gerak
        
        w, h = 30, 10  # ukuran kotak
        cx, cy = self.position.x, self.position.y  # posisi tengah
        
        cos_a = math.cos(angle)  # hitung komponen x
        sin_a = math.sin(angle)  # hitung komponen y
        
        points = []
        for dx, dy in [(-w/2, -h/2), (w/2, -h/2), (w/2, h/2), (-w/2, h/2)]:
            x = cx + dx * cos_a - dy * sin_a  # rotasi titik x
            y = cy + dx * sin_a + dy * cos_a  # rotasi titik y
            points.extend((x, y))
        
        self.canvas.coords(self.id, *points)  # update posisi kotak

    def check_edges(self):
        if self.position.x > self.width:  # jika keluar kanan
            self.position.x = 0           # muncul di kiri
        elif self.position.x < 0:         # jika keluar kiri
            self.position.x = self.width   # muncul di kanan
            
        if self.position.y > self.height:  # jika keluar bawah
            self.position.y = 0            # muncul di atas
        elif self.position.y < 0:          # jika keluar atas
            self.position.y = self.height  # muncul di bawah

# Setup Tkinter
root = tk.Tk()
root.title("Example 3.3: Pointing in the Direction of Motion")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

mover = Mover(canvas, WIDTH, HEIGHT)

def animate():
    canvas.delete("text")
    mouse_x = canvas.winfo_pointerx() - canvas.winfo_rootx()  # posisi mouse x
    mouse_y = canvas.winfo_pointery() - canvas.winfo_rooty()  # posisi mouse y

    mover.update(mouse_x, mouse_y)  # update posisi kotak
    mover.check_edges()             # cek tepi layar
    mover.display()                 # gambar ulang
    
    # Tampilkan teks posisi dan kecepatan
    canvas.create_text(10, 10, anchor="nw",
                       text=f"Pos: ({mover.position.x:.1f}, {mover.position.y:.1f})",
                       font=("Arial", 12), tag="text")
    canvas.create_text(10, 30, anchor="nw",
                       text=f"Vel: ({mover.velocity.x:.2f}, {mover.velocity.y:.2f})",
                       font=("Arial", 12), tag="text")

    root.after(30, animate)  # ulangi setiap 30ms

animate()
root.mainloop()


In [ ]:
# SCRIPT 6 : Simulasi Konversi Koordinat Polar ke Kartesian
import tkinter as tk
import math

# === Setup Tkinter ===
root = tk.Tk()
root.title("Example 3.4: Polar to Cartesian")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Inisialisasi variabel polar ===
r = HEIGHT * 0.45     # Jari-jari (radius)
theta = 0             # Sudut awal

# Objek lingkaran dan garis
circle_id = canvas.create_oval(0, 0, 0, 0, fill="gray", outline="black", width=2)
line_id = canvas.create_line(0, 0, 0, 0, fill="black", width=2)

def animate():
    global theta

    canvas.delete("text")
    canvas.configure(bg="white")

    # Konversi polar ke kartesian
    x = r * math.cos(theta)
    y = r * math.sin(theta)

    # Pusat koordinat di tengah canvas
    center_x = WIDTH / 2
    center_y = HEIGHT / 2

    # Koordinat akhir (x, y) ditranslasi ke posisi canvas
    px = center_x + x
    py = center_y + y

    # Update garis dari pusat ke titik
    canvas.coords(line_id, center_x, center_y, px, py)

    # Update lingkaran di posisi (px, py)
    radius = 24
    canvas.coords(circle_id, px - radius, py - radius, px + radius, py + radius)

    # Tampilkan data
    canvas.create_text(10, 10, anchor="nw", text=f"theta = {theta:.2f} rad", font=("Arial", 12), tag="text")
    canvas.create_text(10, 30, anchor="nw", text=f"x = {x:.1f}", font=("Arial", 12), tag="text")
    canvas.create_text(10, 50, anchor="nw", text=f"y = {y:.1f}", font=("Arial", 12), tag="text")

    # Update sudut theta untuk rotasi
    theta += 0.02

    root.after(30, animate)

animate()
root.mainloop()


In [ ]:
# SCRIPT 7 : Simulasi jalur spiral menggunakan koordinat polar
import tkinter as tk
import math

# Setup Tkinter
root = tk.Tk()
root.title("Exercise 3.4: Spiral Path")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Inisialisasi polar ===
r = 0             # Radius mulai dari 0 (untuk spiral)
theta = 0         # Sudut awal
dot_radius = 5    # Ukuran titik spiral

# Simpan semua titik spiral
spiral_points = []

def animate():
    global theta, r

    # Konversi polar ke kartesian
    x = r * math.cos(theta)
    y = r * math.sin(theta)

    # Geser titik ke tengah canvas
    px = WIDTH / 2 + x
    py = HEIGHT / 2 + y

    # Simpan titik spiral
    spiral_points.append((px, py))

    # Gambar semua titik spiral
    for px, py in spiral_points:
        canvas.create_oval(px - dot_radius, py - dot_radius,
                           px + dot_radius, py + dot_radius,
                           fill="black", outline="")

    # Tambah sudut dan radius
    theta += 0.1   # makin besar = makin cepat berputar
    r += 0.3        # makin besar = spiral makin melebar

    # #Info teks
    # canvas.create_text(10, 10, anchor="nw",
    #                    text=f"theta: {theta:.2f} | radius: {r:.2f}",
    #                    font=("Arial", 10), fill="blue", tag="info")

    root.after(30, animate)

animate()
root.mainloop()


In [ ]:
# SCRIPT 8 : Simulasi kapal luar angkasa yang bergerak dan berputar
import tkinter as tk
import math
from vector import Vector

class Spaceship:
    def __init__(self, canvas, width, height):
        self.canvas = canvas
        self.width = width
        self.height = height

        self.position = Vector(width / 2, height / 2)
        self.velocity = Vector(0, 0)
        self.acceleration = Vector(0, 0)

        self.damping = 0.995
        self.topspeed = 6
        self.heading = 0
        self.r = 16
        self.thrusting = False

        self.id = canvas.create_polygon(0, 0, 0, 0, 0, 0, fill="gray", outline="black")

    def update(self):
        self.velocity.add(self.acceleration)
        self.velocity.mult(self.damping)
        self.velocity.limit(self.topspeed)
        self.position.add(self.velocity)
        self.acceleration = Vector(0, 0)

    def applyForce(self, force):
        self.acceleration.add(force)

    def turn(self, angle):
        self.heading += angle

    def thrust(self):
        angle = self.heading - math.pi / 2  # Kapal digambar tegak
        force = Vector.fromAngle(angle)
        force.mult(0.1)
        self.applyForce(force)
        self.thrusting = True

    def wrapEdges(self):
        buffer = self.r * 2
        if self.position.x > self.width + buffer:
            self.position.x = -buffer
        elif self.position.x < -buffer:
            self.position.x = self.width + buffer

        if self.position.y > self.height + buffer:
            self.position.y = -buffer
        elif self.position.y < -buffer:
            self.position.y = self.height + buffer

    def show(self):
        # Posisi dan rotasi segitiga
        angle = self.heading
        cx, cy = self.position.x, self.position.y

        cos_a = math.cos(angle)
        sin_a = math.sin(angle)

        # Titik segitiga (tubuh kapal)
        points = []
        for dx, dy in [(-self.r, self.r), (0, -self.r), (self.r, self.r)]:
            x = cx + dx * cos_a - dy * sin_a
            y = cy + dx * sin_a + dy * cos_a
            points.extend((x, y))

        self.canvas.coords(self.id, *points)

        # Warna berdasarkan thrust
        if self.thrusting:
            self.canvas.itemconfig(self.id, fill="red")
        else:
            self.canvas.itemconfig(self.id, fill="gray")
        self.thrusting = False

        # Garis arah dari ujung segitiga ke depan
        nose_x = cx + 0 * cos_a - (-self.r) * sin_a
        nose_y = cy + 0 * sin_a + (-self.r) * cos_a

        line_length = 20
        dir_x = math.cos(self.heading - math.pi / 2) * line_length
        dir_y = math.sin(self.heading - math.pi / 2) * line_length

        # Buat atau perbarui garis
        if not hasattr(self, 'direction_line'):
            self.direction_line = self.canvas.create_line(nose_x, nose_y,
                                                          nose_x + dir_x, nose_y + dir_y,
                                                          fill='blue', width=2)
        else:
            self.canvas.coords(self.direction_line,
                               nose_x, nose_y,
                               nose_x + dir_x, nose_y + dir_y)


# Setup Tkinter
root = tk.Tk()
root.title("Exercise 3.5: Asteroids Spaceship")

WIDTH, HEIGHT = 640, 240
canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

ship = Spaceship(canvas, WIDTH, HEIGHT)

# Event state
keys = {"left": False, "right": False, "z": False}

def key_press(event):
    if event.keysym == "Left":
        keys["left"] = True
    elif event.keysym == "Right":
        keys["right"] = True
    elif event.keysym.lower() == "z":
        keys["z"] = True

def key_release(event):
    if event.keysym == "Left":
        keys["left"] = False
    elif event.keysym == "Right":
        keys["right"] = False
    elif event.keysym.lower() == "z":
        keys["z"] = False

root.bind("<KeyPress>", key_press)
root.bind("<KeyRelease>", key_release)

def animate():
    canvas.delete("text")

    if keys["left"]:
        ship.turn(-0.03)
    if keys["right"]:
        ship.turn(0.03)
    if keys["z"]:
        ship.thrust()

    ship.update()
    ship.wrapEdges()
    ship.show()

    canvas.create_text(10, 10, anchor="nw",
                       text=f"Pos: ({ship.position.x:.1f}, {ship.position.y:.1f})",
                       font=("Arial", 12), tag="text")

    root.after(30, animate)

animate()
root.mainloop()


In [ ]:
# SCRIPT 9 : # Simulasi Gerakan Harmonik Sederhana (SHM) Berdasarkan Frame Count
import tkinter as tk
import math

class SHMFrameCountApp:
    def __init__(self, root):
        self.root = root
        self.root.title("SHM - Based on Frame Count")

        self.canvas_width = 640
        self.canvas_height = 240
        self.canvas = tk.Canvas(root, width=self.canvas_width, height=self.canvas_height, bg="white")
        self.canvas.pack()

        self.period = 120
        self.amplitude = 200
        self.frame_count = 0

        self.center_x = self.canvas_width // 2
        self.center_y = self.canvas_height // 2

        # Buat elemen awal (garis dan lingkaran)
        self.line = self.canvas.create_line(self.center_x, self.center_y, self.center_x, self.center_y, width=2)
        self.circle = self.canvas.create_oval(0, 0, 0, 0, fill="gray", outline="black", width=2)

        self.update()

    def update(self):
        # Hitung posisi berdasarkan rumus SHM dengan frameCount
        x = self.amplitude * math.sin((2 * math.pi * self.frame_count) / self.period)
        y = 0  # tetap di sumbu horizontal

        # Update garis dari pusat ke titik bola
        self.canvas.coords(
            self.line,
            self.center_x, self.center_y,
            self.center_x + x, self.center_y + y
        )

        # Update lingkaran (bola)
        r = 24
        self.canvas.coords(
            self.circle,
            self.center_x + x - r,
            self.center_y + y - r,
            self.center_x + x + r,
            self.center_y + y + r
        )

        self.frame_count += 1
        self.root.after(16, self.update)  # sekitar 60 FPS

if __name__ == "__main__":
    root = tk.Tk()
    app = SHMFrameCountApp(root)
    root.mainloop()


In [ ]:
# SCRIPT 10 : Simulasi Gerak Harmonik Sederhana (Simple Harmonic Motion)
import tkinter as tk
import math

class SimpleHarmonicMotionApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Simple Harmonic Motion - Tkinter")
        self.canvas_width = 640
        self.canvas_height = 240
        self.canvas = tk.Canvas(root, width=self.canvas_width, height=self.canvas_height, bg="white")
        self.canvas.pack()

        self.angle = 0
        self.angle_velocity = 0.05
        self.amplitude = 200

        self.center_x = self.canvas_width // 2
        self.center_y = self.canvas_height // 2

        # Garis dan lingkaran
        self.line = self.canvas.create_line(self.center_x, self.center_y, self.center_x, self.center_y, width=2)
        self.circle = self.canvas.create_oval(0, 0, 0, 0, fill="gray", outline="black", width=2)

        self.update()

    def update(self):
        # Hitung posisi berdasarkan gerak harmonik sederhana
        x = self.amplitude * math.sin(self.angle)
        y = 0  # tetap di sumbu horizontal

        # Update garis
        self.canvas.coords(
            self.line,
            self.center_x, self.center_y,
            self.center_x + x, self.center_y + y
        )

        # Update lingkaran
        r = 24
        self.canvas.coords(
            self.circle,
            self.center_x + x - r,
            self.center_y + y - r,
            self.center_x + x + r,
            self.center_y + y + r
        )

        # Update sudut
        self.angle += self.angle_velocity

        # Loop
        self.root.after(16, self.update)  # sekitar 60 FPS

if __name__ == "__main__":
    root = tk.Tk()
    app = SimpleHarmonicMotionApp(root)
    root.mainloop()


In [2]:
# SCRIPT 10 : Simulasi Gerakan Osilator
import tkinter as tk
import math
import random
from vector import Vector

# === Konstanta Layar ===
WIDTH, HEIGHT = 640, 240

# === Kelas Oscillator ===
class Oscillator:
    def __init__(self, canvas):
        self.canvas = canvas
        self.angle = Vector(0, 0)
        self.angle_velocity = Vector(random.uniform(-0.05, 0.05), random.uniform(-0.05, 0.05))
        self.amplitude = Vector(random.uniform(20, WIDTH / 2), random.uniform(20, HEIGHT / 2))

        # Buat objek visual (garis dan lingkaran)
        self.line_id = canvas.create_line(0, 0, 0, 0, fill="black", width=2)
        self.circle_id = canvas.create_oval(0, 0, 0, 0, fill="gray", outline="black")

    def update(self):
        self.angle.add(self.angle_velocity)

    def display(self):
        # Konversi sudut sinusoidal ke posisi (x, y)
        x = math.sin(self.angle.x) * self.amplitude.x
        y = math.sin(self.angle.y) * self.amplitude.y

        center_x, center_y = WIDTH / 2, HEIGHT / 2
        px, py = center_x + x, center_y + y

        # Update garis dari pusat ke titik osilasi
        self.canvas.coords(self.line_id, center_x, center_y, px, py)

        # Update lingkaran
        radius = 16
        self.canvas.coords(
            self.circle_id,
            px - radius, py - radius,
            px + radius, py + radius
        )

# === Setup Tkinter ===
root = tk.Tk()
root.title("Example 3.7: Oscillator Objects")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Buat banyak oscillator ===
oscillators = [Oscillator(canvas) for _ in range(10)]

# === Fungsi animasi utama ===
def animate():
    for osc in oscillators:
        osc.update()
        osc.display()
    root.after(30, animate)

animate()
root.mainloop()


In [1]:
# SCRIPT 10 : Simulasi gelombang statis menggunakan sinus
import tkinter as tk
import math

# === Konstanta Layar ===
WIDTH, HEIGHT = 640, 240
ANGLE_VELOCITY = 0.2
AMPLITUDE = 100
SPACING = 24
RADIUS = 24

# === Setup Tkinter ===
root = tk.Tk()
root.title("Example 3.8: Static Wave")

canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
canvas.pack()

# === Gambar lingkaran gelombang ===
angle = 0
for x in range(0, WIDTH + 1, SPACING):
    # 1) Hitung posisi y menggunakan sinus
    y = AMPLITUDE * math.sin(angle)
    center_y = y + HEIGHT / 2

    # 2) Gambar lingkaran di posisi (x, center_y)
    canvas.create_oval(
        x - RADIUS, center_y - RADIUS,
        x + RADIUS, center_y + RADIUS,
        fill="gray", outline="black", width=2
    )

    # 3) Tambahkan sudut
    angle += ANGLE_VELOCITY

root.mainloop()


In [2]:
import tkinter as tk
import math
from vector import Vector

# Konfigurasi jendela dan kanvas
WIDTH, HEIGHT = 640, 240
STEP = 24         # Jarak antar titik gelombang
CIRCLE_SIZE = 48  # Diameter lingkaran

class WaveSimulation:
    def __init__(self, root):
        self.canvas = tk.Canvas(root, width=WIDTH, height=HEIGHT, bg="white")
        self.canvas.pack()

        self.start_angle = 0.0       # Mirip variabel startAngle di Processing
        self.angle_velocity = 0.2    # Kecepatan perubahan sudut antar titik

        self.circles = []  # List untuk menyimpan ID lingkaran di canvas

        # Inisialisasi lingkaran pada posisi awal
        for x in range(0, WIDTH + 1, STEP):
            y = HEIGHT // 2  # Posisi y sementara
            circle_id = self.canvas.create_oval(
                x - CIRCLE_SIZE//2, y - CIRCLE_SIZE//2,
                x + CIRCLE_SIZE//2, y + CIRCLE_SIZE//2,
                fill="gray", outline="black"
            )
            self.circles.append(circle_id)

        # Mulai animasi
        self.animate()

    def animate(self):
        # Bersihkan background dengan cara mengganti warna semua oval
        self.canvas.delete("all")

        angle = self.start_angle
        self.start_angle += 0.02  # Seiring waktu sudut awal bertambah

        # Gambar ulang semua lingkaran mengikuti pola gelombang
        for i, x in enumerate(range(0, WIDTH + 1, STEP)):
            # Ubah nilai sin menjadi nilai y (0 - HEIGHT)
            sin_value = math.sin(angle)
            y = self.map_value(sin_value, -1, 1, 0, HEIGHT)

            # Gambar ulang lingkaran
            circle_id = self.canvas.create_oval(
                x - CIRCLE_SIZE//2, y - CIRCLE_SIZE//2,
                x + CIRCLE_SIZE//2, y + CIRCLE_SIZE//2,
                fill="gray", outline="black"
            )

            angle += self.angle_velocity

        # Ulangi fungsi animate setiap 30 milidetik (~33 FPS)
        self.canvas.after(30, self.animate)

    @staticmethod
    def map_value(value, start1, stop1, start2, stop2):
        # Fungsi seperti map() di Processing: ubah nilai dari satu rentang ke rentang lain
        return start2 + (stop2 - start2) * ((value - start1) / (stop1 - start1))


if __name__ == "__main__":
    root = tk.Tk()
    root.title("Gelombang Sine dengan Tkinter")
    sim = WaveSimulation(root)
    root.mainloop()
